# Imports

In [5]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [6]:
np.random.seed(131)
initial_params = np.random.random([165])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [7]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [8]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label == 0:
        x.append(example)
        y.append(-1)
    elif label == 1:
        x.append(example)
        y.append(1)

In [9]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [10]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [11]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [12]:
device = qml.device("default.qubit", wires=8)

In [13]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[1, 2])
    ArbitraryUnitary(params[15:30], wires=[3, 4])
    ArbitraryUnitary(params[30:45], wires=[5, 6])

    # Second layer
    ArbitraryUnitary(params[45:60], wires=[0, 1])
    ArbitraryUnitary(params[60:75], wires=[2, 3])
    ArbitraryUnitary(params[75:90], wires=[4, 5])
    ArbitraryUnitary(params[90:105], wires=[6, 7])

    # Third layer
    ArbitraryUnitary(params[105:120], wires=[2, 5])

    # Fourth layer
    ArbitraryUnitary(params[120:135], wires=[1, 2])
    ArbitraryUnitary(params[135:150], wires=[5, 6])

    # Fifth layer
    ArbitraryUnitary(params[150:165], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [14]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549
 0.84869439 0.53066622 0.49383494 0.65230764 0.22667618 0.61716246
 0.2005521  0.49093199 0.94852991 0.98197027 0.17315162 0.74251972
 0.38646803 0.65110712 0.7434636  0.24777039 0.49229597 0.42474983
 0.29006936 0.98668888 0.33945016 0.1997036

# Accuracy test definition

In [15]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [16]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5563139931740615
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.8532423208191127
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.8976109215017065
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.8646188850967008
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.8930602957906713
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.8953356086461889
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.8828213879408419
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.9328782707622298
best accuracy so far!
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 

epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.9385665529010239
test_accuracies: [0.8828213879408419, 0.9328782707622298, 0.9408418657565415, 0.9215017064846416, 0.9340159271899886, 0.9397042093287827, 0.9203640500568828, 0.9203640500568828, 0.9362912400455062, 0.9385665529010239, 0.9124004550625711, 0.9453924914675768, 0.9351535836177475, 0.9306029579067122, 0.9306029579067122, 0.9351535836177475, 0.9328782707622298, 0.9306029579067122, 0.9283276450511946, 0.9271899886234357, 0.9397042093287827, 0.9397042093287827, 0.9283276450511946, 0.931740614334471, 0.9294653014789533, 0.9203640500568828, 0.9237770193401593, 0.9397042093287827, 0.9408418657565415, 0.9385665529010239]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.9397042093287827
test_accuracies: [0.9328782707622298, 0.9408418657565415, 0.9215017064846416, 0.9340159271899886, 0.9397042093287827, 0.9203640500568828, 0.9203640500568828, 0.936291240045506

epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.9419795221843004
test_accuracies: [0.9453924914675768, 0.9351535836177475, 0.9306029579067122, 0.9306029579067122, 0.9351535836177475, 0.9328782707622298, 0.9306029579067122, 0.9283276450511946, 0.9271899886234357, 0.9397042093287827, 0.9397042093287827, 0.9283276450511946, 0.931740614334471, 0.9294653014789533, 0.9203640500568828, 0.9237770193401593, 0.9397042093287827, 0.9408418657565415, 0.9385665529010239, 0.9397042093287827, 0.9385665529010239, 0.9374288964732651, 0.9283276450511946, 0.9089874857792947, 0.9306029579067122, 0.9294653014789533, 0.9408418657565415, 0.9397042093287827, 0.9419795221843004, 0.9419795221843004]


In [17]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 9.73508724e-01  4.58678162e-01  6.22504283e-01  2.08925123e-02
  1.82716288e-01  8.63799516e-02  8.71627614e-01 -1.28700016e-01
  2.09587317e-01  1.37187170e+00  6.32691314e-01  6.94875458e-01
 -1.01687243e-01  5.15166548e-01  5.19198257e-01 -5.53509363e-01
  8.51664495e-01 -5.55249003e-02  8.33121251e-01 -7.09681966e-01
  7.19659887e-01  6.15271552e-01  1.90456282e-01  7.39609814e-01
 -1.21955434e+00  4.14175507e-01  1.49130847e+00  5.51328854e-01
  1.01355292e+00  5.53453854e-01 -1.75233608e-02  1.79743803e-01
 -2.28536100e-02  9.48731565e-01  3.58614630e-01  5.14262889e-01
  3.43990053e-01  2.25270978e-01  2.61077466e-01  1.22324308e+00
  5.02457552e-01  6.56720156e-02  1.24662686e+00  6.75062087e-01
  1.36732349e+00  1.51525235e+00  1.13663602e+00  5.84296294e-01
  3.97529435e-01  4.28476164e-01  1.28681670e+00  8.21555998e-01
  6.85414796e-02  8.84313589e-01  4.46943995e-01  1.86695861e-01
 -6.34059528e-01  1.25548663e+00  6.63208118e-01  1.25157968e+0

# Testing

In [18]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.9341452413170952


In [19]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 2.8708736896514893
training time: 4811.489710092545
test time: 190.14241743087769
total time: 5004.503001213074
